In [ ]:
# | default_exp modeling_options_utils

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
import random
import shutil
import datetime
import collections
import numpy as np
import pandas as pd
from typing import Dict
from pathlib import Path, PosixPath
from pydantic import ValidationError
from pysureau.pysureau_utils import dict_to_csv
from pysureau.parameter_validators import ModelingOptionsParameterValidator
from pysureau.plant_utils import read_vegetation_file

In [ ]:
# | export


def create_empty_modeling_options_file(
    path: Path,  # Path to the folder where the parameter files will be saved. If set to None then the files will be saved at the current working directory
) -> Dict:  # Return two dictionary file for user input
    "Function for creating the CSV templates necessary for the modeling options file"

    # Assert parameters ---------------------------------------------------------
    assert isinstance(path, str) | isinstance(path, PosixPath), (
        f'Input path must be a str, not a {type(path).__name__}'
    )

    # Convert string to Path if provided ----------------------------------------
    path = Path(path)

    # Main
    if os.path.exists(path):
        modeling_options_params = {
            'eord': 1,
            'lcav': 1,
            'scav': 1,
            'latitude': 'NA',
            'longitude': 'NA',
            'year_end': 'NA',
            'year_start': 'NA',
            'time_step_for_evapo': 1,
            'threshold_mortality': 90,
            'transpiration_granier_arg_c': 0,
            'custom_small_time_step_in_sec': 600,
            'transpiration_granier_arg_b': 0.134,
            'transpiration_granier_arg_a': -0.006,
            'print_prog': True,
            'defoliation': False,
            'constant_climate': False,
            'output_overwrite': False,
            'output_type': 'simple',
            'pet_formulation': 'pt',
            'rn_formulation': 'linacre',
            'numerical_scheme': 'implicit',
            'resolution_output': 'subdaily',
            'transpiration_model': 'jarvis',
            'comp_options_for_evapo': 'normal',
            'stomatal_reg_formulation': 'sigmoid',
            'output_path': '~/pysureau_project_EDITME/2_model_outputs',
        }
        # Write to CSV file
        dict_to_csv(
            path=path,
            filename='modeling_options.csv',
            dictionary=modeling_options_params,
        )

    else:
        raise ValueError('Failed creating empty modeling options file')

In [ ]:
# | export


def read_modeling_options_file(
    file_path: Path,  # Path to the sureau_parameter_files folder containing the csv files with parameter values i.e path/to/sureau_parameter_files/file_name.csv
    vegetation_param_file_path: Path,
    sep: str = ',',  # CSV file separator can be ',' or ';'
) -> Dict:
    "Create a dictionary containing modeling options that can be used as an input in run.SurEauR"  #

    # Assert parameters ---------------------------------------------------------

    assert os.path.exists(file_path), (
        f'sureau_parameter_files not found at {file_path}. Save the CSV inside the 1_parameter_files folder in the pysureau project'
    )

    assert os.path.exists(vegetation_param_file_path), (
        f'sureau_parameter_files not found at {vegetation_param_file_path}. Save the CSV inside the 1_parameter_files folder in the pysureau project'
    )

    # Read CSV parameter file ---------------------------------------------------
    vegetation_parameters_dict = read_vegetation_file(
        file_path=vegetation_param_file_path
    )
    modelling_options_data = pd.read_csv(file_path, sep=sep, header=0)

    # Validate the column names of soil data are parameter_name, parameter_value
    # To correctly transform the the CSV file as dict later
    if modelling_options_data.columns.tolist() != [
        'parameter_name',
        'parameter_value',
    ]:
        raise ValueError(
            'Column names in modelling options file must be called parameter_name and parameter_value'
        )

    # Validate parameter file ---------------------------------------------------

    # Transform dataframe into dictionary
    modelling_options_dict = modelling_options_data.set_index(
        'parameter_name'
    ).to_dict()['parameter_value']

    # Loop over dictionary to transform the data types
    # If this is not done all values will be considered str

    parameters_of_class_str = [
        'print_prog',
        'defoliation',
        'constant_climate',
        'output_overwrite',
        'output_type',
        'pet_formulation',
        'rn_formulation',
        'numerical_scheme',
        'resolution_output',
        'transpiration_model',
        'comp_options_for_evapo',
        'stomatal_reg_formulation',
        'output_path',
    ]

    parameters_of_class_bool = ['eord', 'lcav', 'scav']

    # Loop over all keys. This step is done beacuse the csv file can contain str
    # and float values
    for each_key in modelling_options_dict.keys():
        if each_key in parameters_of_class_str:
            # If value is in parameters_of_class_str then transform to str
            modelling_options_dict[each_key] = str(
                modelling_options_dict[each_key]
            )

        # Special case. Done in this way beacuse param can be an integer or a str
        elif each_key == 'time_step_for_evapo':
            if modelling_options_dict[each_key] == 'variable':
                modelling_options_dict[each_key] = str(
                    modelling_options_dict[each_key]
                )

            else:
                modelling_options_dict[each_key] = int(
                    modelling_options_dict[each_key]
                )

        elif each_key in parameters_of_class_bool:
            # If value is in parameters_of_class_bool then transform to int
            # Done in this way because values only can be 0 or 1 and if I do as
            # bool(modelling_options_dict[each_key]) instead of
            # int(modelling_options_dict[each_key]) a value of 2 will return
            # True
            modelling_options_dict[each_key] = int(
                modelling_options_dict[each_key]
            )

        else:
            # Transform parameters values to float
            modelling_options_dict[each_key] = float(
                modelling_options_dict[each_key]
            )

    # Validate modelling_options_dict pydanthic schema --------------------------
    try:
        ModelingOptionsParameterValidator.model_validate(modelling_options_dict)

    except ValidationError as error:
        raise (error)

    # Compare end_year_simulation is larger than start_year_simulation
    assert (
        modelling_options_dict['year_start']
        <= modelling_options_dict['year_end']
    ), (
        f'year_start ({modelling_options_dict["year_start"]}) is larger than year_end ({modelling_options_dict["year_end"]})'
    )

    # Validate output path
    if os.path.exists(Path(modelling_options_dict['output_path'])):
        pass

    else:
        raise ValueError(
            f'Output path {Path(modelling_options_dict["output_path"])} not found'
        )

    # Create new parameters based on input parameters ---------------------------

    # Create array with time steps for the evapo

    if modelling_options_dict['time_step_for_evapo'] == 'variable':
        time = np.array([0, 6, 12, 14, 16, 22])
        raise ValueError(
            'time_step_for_evapo set to "variable". This has not been implemented yet'
        )

    else:
        time = np.arange(
            0, 24, modelling_options_dict['time_step_for_evapo'], dtype=int
        )
        modelling_options_dict['time'] = time

    # Create comp_options
    modelling_options_dict['comp_options'] = collections.defaultdict(list)

    # Every 10min, 6min, 3min, 1min
    if modelling_options_dict['comp_options_for_evapo'] == 'normal':
        # Add key value pairs to the comp_dictionary
        modelling_options_dict['comp_options']['numerical_scheme'] = (
            modelling_options_dict['numerical_scheme']
        )
        modelling_options_dict['comp_options']['nsmalltimesteps'] = (
            modelling_options_dict['time_step_for_evapo']
            * np.array([6, 10, 20, 60])
        )
        modelling_options_dict['comp_options']['lsym'] = 1
        modelling_options_dict['comp_options']['ssym'] = 1
        modelling_options_dict['comp_options']['clapo'] = 1
        modelling_options_dict['comp_options']['ctapo'] = 1
        modelling_options_dict['comp_options']['eord'] = modelling_options_dict[
            'eord'
        ]
        modelling_options_dict['comp_options']['lcav'] = modelling_options_dict[
            'lcav'
        ]
        modelling_options_dict['comp_options']['scav'] = modelling_options_dict[
            'scav'
        ]

    # every 10 seconds
    elif modelling_options_dict['comp_options_for_evapo'] == 'accurate':
        modelling_options_dict['comp_options']['numerical_scheme'] = (
            modelling_options_dict['numerical_scheme']
        )
        modelling_options_dict['comp_options']['nsmalltimesteps'] = (
            modelling_options_dict['time_step_for_evapo'] * np.array([600])
        )
        modelling_options_dict['comp_options']['lsym'] = 1
        modelling_options_dict['comp_options']['ssym'] = 1
        modelling_options_dict['comp_options']['clapo'] = 1
        modelling_options_dict['comp_options']['ctapo'] = 1
        modelling_options_dict['comp_options']['eord'] = modelling_options_dict[
            'eord'
        ]
        modelling_options_dict['comp_options']['lcav'] = modelling_options_dict[
            'lcav'
        ]
        modelling_options_dict['comp_options']['scav'] = modelling_options_dict[
            'scav'
        ]

    # every hours, every 10 min
    elif modelling_options_dict['comp_options_for_evapo'] == 'fast':
        modelling_options_dict['comp_options']['numerical_scheme'] = (
            modelling_options_dict['numerical_scheme']
        )
        modelling_options_dict['comp_options']['nsmalltimesteps'] = (
            modelling_options_dict['time_step_for_evapo'] * np.array([1, 6])
        )
        modelling_options_dict['comp_options']['lsym'] = 1
        modelling_options_dict['comp_options']['ssym'] = 1
        modelling_options_dict['comp_options']['clapo'] = 1
        modelling_options_dict['comp_options']['ctapo'] = 1
        modelling_options_dict['comp_options']['eord'] = modelling_options_dict[
            'eord'
        ]
        modelling_options_dict['comp_options']['lcav'] = modelling_options_dict[
            'lcav'
        ]
        modelling_options_dict['comp_options']['scav'] = modelling_options_dict[
            'scav'
        ]

    # every customSmallTimeStepInSec
    elif modelling_options_dict['comp_options_for_evapo'] == 'custom':
        modelling_options_dict['comp_options']['numerical_scheme'] = (
            modelling_options_dict['numerical_scheme']
        )
        modelling_options_dict['comp_options']['nsmalltimesteps'] = (
            modelling_options_dict['time_step_for_evapo'] * 3600
        ) / modelling_options_dict['custom_small_time_step_in_sec']
        modelling_options_dict['comp_options']['lsym'] = 1
        modelling_options_dict['comp_options']['ssym'] = 1
        modelling_options_dict['comp_options']['clapo'] = 1
        modelling_options_dict['comp_options']['ctapo'] = 1
        modelling_options_dict['comp_options']['eord'] = modelling_options_dict[
            '`eord'
        ]
        modelling_options_dict['comp_options']['lcav'] = modelling_options_dict[
            '`lcav'
        ]
        modelling_options_dict['comp_options']['scav'] = modelling_options_dict[
            '`scav'
        ]

    else:
        raise ValueError('Failed creating comp_options_for_evapo')

    # Append parameters to modeling options -------------------------------------

    if modelling_options_dict['transpiration_model'] == 'granier':
        raise ValueError(print('granier option have been not implemented yet'))

        # modelling_options_dict['transpiration_granier_args'] = (
        #    collections.defaultdict(
        #        list,
        #        {
        #            'a': modelling_options_dict['transpiration_granier_arg_a'],
        #            'b': modelling_options_dict['transpiration_granier_arg_b'],
        #            'c': modelling_options_dict['transpiration_granier_arg_c'],
        #        },
        #    )
        # )

    elif modelling_options_dict['transpiration_model'] == 'jarvis':
        modelling_options_dict['transpiration_model_params'] = (
            collections.defaultdict(
                list,
                {
                    'g_crown0': vegetation_parameters_dict['g_crown0'],
                    'gs_max': vegetation_parameters_dict['gs_max'],
                    'gs_night': vegetation_parameters_dict['gs_night'],
                    'jarvis_par': vegetation_parameters_dict['jarvis_par'],
                    't_gs_sens': vegetation_parameters_dict['t_gs_sens'],
                    't_gs_optim': vegetation_parameters_dict['t_gs_optim'],
                },
            )
        )

    else:
        raise ValueError(
            'Failed appending params a,b, c for granier transpiration_model'
        )

    # Foliage traits
    if vegetation_parameters_dict['foliage'] == 'deciduous':
        modelling_options_dict['foliage_params'] = collections.defaultdict(
            list,
            {
                'day_start': vegetation_parameters_dict['day_start'],
                'f_crit': vegetation_parameters_dict['f_crit'],
                'nb_day_lai': vegetation_parameters_dict['nb_day_lai'],
                't_base': vegetation_parameters_dict['t_base'],
            },
        )

    elif vegetation_parameters_dict['foliage'] == 'forced':
        modelling_options_dict['foliage_params'] = collections.defaultdict(
            list,
            {
                'day_start_force': vegetation_parameters_dict['day_start'],
                'day_end_forced': vegetation_parameters_dict['day_end'],
                'nb_day_lai': vegetation_parameters_dict['nb_day_lai'],
            },
        )

    else:
        modelling_options_dict['foliage_params'] = collections.defaultdict(
            list, {}
        )
    modelling_options_dict['stop_simulation_dead_plant'] = (
        modelling_options_dict['print_prog']
    )

    return modelling_options_dict

In [ ]:
# read_modeling_options_file(file_path='/tmp/pysureau_project_dbTPrigW/1_parameter_files/modeling_options.csv',
#                           vegetation_param_file_path = '/tmp/pysureau_project_dbTPrigW/1_parameter_files/vegetation_parameters.csv')


{'eord': 1,
 'lcav': 1,
 'scav': 1,
 'latitude': 90.0,
 'longitude': 100.0,
 'year_end': 1992.0,
 'year_start': 1991.0,
 'time_step_for_evapo': 1,
 'threshold_mortality': 90.0,
 'transpiration_granier_arg_c': 0.0,
 'custom_small_time_step_in_sec': 600.0,
 'transpiration_granier_arg_b': 0.134,
 'transpiration_granier_arg_a': -0.006,
 'print_prog': 'TRUE',
 'defoliation': 'FALSE',
 'constant_climate': 'FALSE',
 'output_overwrite': 'FALSE',
 'output_type': 'simple',
 'pet_formulation': 'pt',
 'rn_formulation': 'linacre',
 'numerical_scheme': 'implicit',
 'resolution_output': 'subdaily',
 'transpiration_model': 'jarvis',
 'comp_options_for_evapo': 'normal',
 'stomatal_reg_formulation': 'sigmoid',
 'output_path': '/tmp/pysureau_project_dbTPrigW/1_parameter_files',
 'time': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23]),
 'comp_options': defaultdict(list,
             {'numerical_scheme': 'implicit',
              'nsmalltimest